In [1]:
API_KEY = '6d4bd60e237eae6df24af2c0189a10278154d4f2079dec475b7d15ca11baea04'
#instantiate query api
from sec_api import QueryApi
queryApi = QueryApi(api_key=API_KEY) 

In [2]:
import sys
# import os 

print(sys.executable)
# print(os.getcwd()) 

D:\coding\Nanik\SEC\backend\venv\scripts\python.exe


In [3]:
import pandas as pd
from time import sleep 
import concurrent.futures
file_path = "D:/coding/Nanik/SEC/backend/sec_app/data/stocks_perf_data.xlsx"
df = pd.read_excel(file_path)

tickers = df['Symbol'].dropna().unique().tolist()

In [4]:
def get_filings(query):
  from_param = 0
  size_param = 200
  all_filings = []

  while True:
    query['from'] = from_param
    query['size'] = size_param

    response = queryApi.get_filings(query)
    filings = response['filings']

    if len(filings) == 0:
      break

    all_filings.extend(filings)

    from_param += size_param

  return pd.json_normalize(all_filings)

In [ ]:
def fetch_filings_for_ticker(ticker):
  form_type_query  = 'formType:("10-K") AND NOT formType:("10-K/A", "10-Q/A", NT)'
  ticker_query     = f'ticker:({ticker})'
  date_range_query = 'filedAt:[2020-01-01 TO 2025-03-15]'
  lucene_query = f"{form_type_query} AND {ticker_query} AND {date_range_query}"

  search_query = {
    "query": lucene_query,
    "from": "0",
    "size": "200",
    "sort": [{ "filedAt": { "order": "desc" } }]
  }

  try:
    filings = get_filings(search_query)  # Assuming `get_filings()` is already defined
    return filings
  except Exception as e:
    print(f"Error fetching {ticker}: {e}")
    return None

    # Iterate over all tickers
all_filings = {}
number_of_workers = 15

with concurrent.futures.ThreadPoolExecutor(max_workers=number_of_workers) as executor:
    future_to_ticker = {executor.submit(fetch_filings_for_ticker, ticker): ticker for ticker in tickers}
    for future in concurrent.futures.as_completed(future_to_ticker):
        ticker = future_to_ticker[future]
        try:
            all_filings[ticker] = future.result()
            print(f"Fetched data for {ticker}")
        except Exception as e:
            print(f"Error occurred for {ticker}: {e}")

print(f"Fetched data for {len(all_filings)} tickers")


Fetched data for AACB
Fetched data for AACG
Fetched data for AAL
Fetched data for AAME
Fetched data for AAM
Fetched data for A
Fetched data for AAMI
Fetched data for AACT
Fetched data for AADI
Fetched data for AA
Fetched data for AAON
Fetched data for AAPG
Fetched data for AAOI
Fetched data for AARD
Fetched data for AAP
Fetched data for ABEV
Fetched data for AB
Fetched data for AAT
Fetched data for AAPL
Fetched data for ABAT
Fetched data for ABBV
Fetched data for ABLV
Fetched data for ABCB
Fetched data for ABEO
Fetched data for ABCL
Fetched data for ABG
Fetched data for ABM
Fetched data for ABTS
Fetched data for ABL
Fetched data for ABOS
Fetched data for ABNB
Fetched data for ABP
Fetched data for ABR
Fetched data for ABT
Fetched data for ABSI
Fetched data for ABVE
Fetched data for ABVX
Fetched data for ABVC
Fetched data for ACB
Fetched data for AC
Fetched data for ABUS
Fetched data for ACA
Fetched data for ACAD
Fetched data for ACCO
Fetched data for ACDC
Fetched data for ACHL
Fetched d

In [ ]:
lucene_query

In [ ]:
search_query

In [ ]:
filings.head(20)

In [ ]:
urls = filings[['ticker',
                'formType',
                'periodOfReport',
                'filedAt',
                'linkToFilingDetails']].rename(columns={'linkToFilingDetails': 'filingUrl'})

urls['financialReportsUrl'] = urls['filingUrl'].apply(lambda url: '/'.join(url.split('/')[:-1]) + '/Financial_Report.xlsx')

urls.head(10)

In [ ]:
#Download Financial_reports.xlsx of All EDGAR Filings
import os, requests

folder_path = './reports'
API_KEY = '6d4bd60e237eae6df24af2c0189a10278154d4f2079dec475b7d15ca11baea04'
if not os.path.exists(folder_path):
  os.makedirs(folder_path)


def download_report(filing,api_key):
    try:
      reports_path = filing['financialReportsUrl'].replace('https://www.sec.gov/Archives/edgar/data/', '')
      base_url = f'https://archive.sec-api.io/{reports_path}'
      render_api_url = f'{base_url}?token={api_key}'

      response = requests.get(render_api_url)
      response.raise_for_status()
      print(f"Requesting: {render_api_url}")
      print(f"Response Code: {response.status_code}, Response Text: {response.text}")


      file_name = f"{filing['ticker']}-{filing['periodOfReport']}-{filing['formType']}.xlsx"
      file_path = f"{folder_path}/{file_name}"

      with open(file_path, 'wb') as output:
              output.write(response.content)
    except Exception as e:
          print(f"Error downloading report for {filing['ticker']}: {e}")
          

In [ ]:
#!pip install -q pandarallel

# from pandarallel import pandarallel
# number_of_workers = 4
# pandarallel.initialize(progress_bar=False, nb_workers=number_of_workers, verbose=0)

In [ ]:
import concurrent.futures
import time

def download_with_rate_limit(filing, api_key):
    """Download a report with rate limiting to avoid API limits"""
    try:
        download_report(filing, api_key)
    except requests.exceptions.RequestException as e:
        print(f"Request Error for {filing['ticker']}: {e}")
        time.sleep(2)  # Wait before retrying
        download_report(filing, api_key)

number_of_workers = 15

with concurrent.futures.ThreadPoolExecutor(max_workers=number_of_workers) as executor:
    futures = [executor.submit(download_with_rate_limit, filing, API_KEY) for _, filing in urls.iterrows()]
    for future in concurrent.futures.as_completed(futures):
        try:
            future.result()
        except Exception as e:
            print(f"Error occurred: {e}")

print(f"Downloaded {len(urls)} reports")

In [ ]:
with open("backend/sec_app/sec_finance_data_excel.ipynb", "r", encoding="utf-8", errors="replace") as f:
    content = f.read()

with open("backend/sec_app/sec_finance_data_excel_fixed.ipynb", "w", encoding="utf-8") as f:
    f.write(content)

print("Fixed notebook saved as sec_finance_data_excel_fixed.ipynb")
